# OpenClassrooms - Ingenieur IA
# Projet 9 - My Content - Partie 3 : Préparation au déploiement
# Réalisez une application mobile de recommandation de contenu

## Objectif du projet : 
- **Réalisez une application mobile de recommandation de contenu**

## Plan :
- **Partie 1 : Analyse et préparation des données**
    - Chargement des données
    - Analyse Exploratoire des Données
    - Nettoyage des données
    - Séparation des données
    - Sauvegarde des données préparées
        
    
- **Partie 2 : Modélisation et évaluation**
    - Baseline recommandation
        - Modèle basé sur la popularité des articles
    - Content Based recommandation
        - Modèle basé sur la catégorie des articles
        - Modèle basé sur la similarité entre les articles
    - Collaborative Filtering recommandation
        - Model based
            - ALS Model
            - BPR Model
            - LMF Model
        - Memory based
            - Nearest Neighbours with Cosine Recommender Model
    - Récapitulatif et choix du meilleur modèle
    
    
- **Partie 3 : Préparation au déploiement**
    - Récupération du Workpsace Azure
    - Identifications des éléments
    - Enregistrement des données nécessaires au déploiement dans Azure

# Partie 3 : Préparation au déploiement

In [2]:
import os
import numpy as np
import pandas as pd
from azureml.core import Workspace
from azure.storage.blob import BlobServiceClient
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference

## Récupération du Workpsace Azure
- Authentification utilisant le service sécurisé et recommandé **'Service Principal Authentication'**

In [3]:
svc_pr_password = os.environ['BOOKSHELF_SPA_PASSWORD']
tenant_id = '894ad120-c276-4dfa-b218-d82b3fece6a7'
application_id = '52dfe3c3-0fcc-47d0-bc76-0429bf40abc7'

In [4]:
svc_pr = ServicePrincipalAuthentication(
        tenant_id=tenant_id,
        service_principal_id=application_id,
        service_principal_password=svc_pr_password)

In [5]:
subscription_id = subscription_id = os.environ['AZURE_SUBSCRIPTION_ID']
resource_group = 'p9bookshelffunctionapp'
workspace_name = 'WS-IA-P9'
ws = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        workspace_name=workspace_name,
        auth=svc_pr)

In [6]:
print(ws.name, ws.location, ws.resource_group, sep='\t')

WS-IA-P9	francecentral	p9bookshelffunctionapp


## Identifications des éléments

In [7]:
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [8]:
blob_datastore_name='datastore_bookshelf'

In [9]:
account_name=os.getenv("AZURE_ACCOUNT_NAME")
account_key=os.getenv("AZURE_ACCOUNT_KEY")

## Enregistrement des données nécessaires au déploiement dans Azure

### Création du Container Azure

In [10]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_name = "bookshelf-data"
container_client = blob_service_client.create_container(container_name)

### Création du Datastore Azure

In [11]:
blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

### Récupération du Datastore Azure

In [12]:
datastore = Datastore.get(ws, blob_datastore_name)

In [13]:
print(datastore.datastore_type, datastore.account_name)

AzureBlob wsiap91720168797


### Préparation des données pour la déploiement
- On garde uniquement les données nécessaires au fonctionnemet du modèle déployé afin d'optimiser le fonctionnement de l'application

In [14]:
df_data_train = pd.read_csv('bookshelf_dataset_train.csv')

In [15]:
df_data_train

,user_id,click_timestamp,click_country,click_region,article_id,category_id,words_count,click_datetime
0,59,1506826800026,1,21,234853,375,140,2017-10-01 03:00:00.026
1,154,1506826804207,1,25,96663,209,206,2017-10-01 03:00:04.207
2,111,1506826814140,1,9,202436,327,333,2017-10-01 03:00:14.140
3,133,1506826823065,1,13,284847,412,266,2017-10-01 03:00:23.065
4,59,1506826830026,1,21,234995,375,155,2017-10-01 03:00:30.026
...,...,...,...,...,...,...,...,...
1033991,119210,1507506684388,1,20,233658,375,149,2017-10-08 23:51:24.388
1033992,235646,1507506685571,1,25,129434,252,168,2017-10-08 23:51:25.571
1033993,2504,1507506686115,1,25,202632,327,189,2017-10-08 23:51:26.115
1033994,25029,1507506686811,1,25,272202,399,206,2017-10-08 23:51:26.811


In [16]:
columns_to_drop = ['click_timestamp', 'click_country', 'click_region', 'words_count', 'click_datetime'] 

In [17]:
df_data_clean = df_data_train.drop(columns=columns_to_drop)

In [18]:
df_data_clean

,user_id,article_id,category_id
0,59,234853,375
1,154,96663,209
2,111,202436,327
3,133,284847,412
4,59,234995,375
...,...,...,...
1033991,119210,233658,375
1033992,235646,129434,252
1033993,2504,202632,327
1033994,25029,272202,399


### Sauvegarde des données préparées pour le déploiement

In [19]:
df_data_clean.to_csv("bookshelf_dataset_clean.csv", index=False)

### Upload des données préparées pour le déploiement dans le DataStore

In [20]:
datastore.upload_files(['./bookshelf_dataset_clean.csv'], target_path='/bookshelf', overwrite=True, show_progress=True)

Uploading an estimated of 1 files
Uploading ./bookshelf_dataset_clean.csv
Uploaded ./bookshelf_dataset_clean.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_44f356717f2545bdb2257dc0a8e30183